In [1]:
import Data.List
import qualified Data.Map as Map
import Data.Semigroup
import Data.Maybe

In [2]:
file1 :: [(Int,Double)]
file1 = [ (1, 200.1), (2, 199.5), (3, 199.4)
    , (4, 198.9), (5, 199.0), (6, 200.2)
    , (9, 200.3), (10, 201.2), (12, 202.9)]

file2 :: [(Int,Double)]
file2 = [(11, 201.6), (12, 201.5), (13, 201.5)
    ,(14, 203.5), (15, 204.9), (16, 207.1)
    ,(18, 210.5), (20, 208.8)]

file3 :: [(Int,Double)]
file3 = [(10, 201.2), (11, 201.6), (12, 201.5)
    ,(13, 201.5), (14, 203.5), (17, 210.5)
    ,(24, 215.1), (25, 218.7)]

file4 :: [(Int,Double)]
file4 = [(26, 219.8), (27, 220.5), (28, 223.8)
    ,(29, 222.8), (30,223.8), (31, 221.7)
    ,(32, 222.3), (33,220.8), (34, 219.4)
    ,(35, 220.1), (36,220.6)]




In [3]:
minimum [1, 2]

1

In [4]:
data TimeSeries a = TimeSeries [Int] [Maybe a] deriving Show

In [5]:
createTimeSeries :: [Int] -> [a] -> TimeSeries a
createTimeSeries keys values = TimeSeries contiguousKeys contiguousValues
    where           
        contiguousKeys = [minimum keys .. maximum keys]
        tmpMap = Map.fromList (zip keys values)
        contiguousValues = [Map.lookup key tmpMap | key <- contiguousKeys]
        
fileToTimeSeries :: [(Int, a)] -> TimeSeries a
fileToTimeSeries x = createTimeSeries keys values
    where (keys, values) = unzip x 

In [6]:
fileToTimeSeries file1

TimeSeries [1,2,3,4,5,6,7,8,9,10,11,12] [Just 200.1,Just 199.5,Just 199.4,Just 198.9,Just 199.0,Just 200.2,Nothing,Nothing,Just 200.3,Just 201.2,Nothing,Just 202.9]

In [7]:
showPair key (Just value) = show key ++ "|" ++ show value ++ "\n"
showPair key Nothing = show key ++ "|" ++ "NA" ++ "\n"

instance Show a => Show (TimeSeries a) where
    show (TimeSeries keys values) = mconcat rows
        where rows = zipWith showPair keys values
        
fileToTimeSeries file1

1|200.1
2|199.5
3|199.4
4|198.9
5|199.0
6|200.2
7|NA
8|NA
9|200.3
10|201.2
11|NA
12|202.9

In [8]:
ts1 = fileToTimeSeries file1
ts2 = fileToTimeSeries file2
ts3 = fileToTimeSeries file3
ts4 = fileToTimeSeries file4
ts4

26|219.8
27|220.5
28|223.8
29|222.8
30|223.8
31|221.7
32|222.3
33|220.8
34|219.4
35|220.1
36|220.6

In [36]:
insertMaybe :: Ord k => Map.Map k v -> (k, Maybe v) -> Map.Map k v
insertMaybe m (_, Nothing) = m
insertMaybe m (k, (Just val)) = Map.insert k val m

In [66]:
-- mergeTimeSeries :: TimeSeries a -> TimeSeries a -> TimeSeries a
mergeTimeSeries (TimeSeries ak av) (TimeSeries bk bv) = TimeSeries timeAxis result
    where
        fullAxis = ak ++ bk
        timeAxis = [minimum fullAxis .. maximum fullAxis]
        tmpMap = foldl insertMaybe Map.empty (zip ak av)
        updatedMap = foldl insertMaybe tmpMap (zip bk bv)
        result = [Map.lookup key updatedMap | key <- timeAxis]
mergeTimeSeries ts1 ts2

1|200.1
2|199.5
3|199.4
4|198.9
5|199.0
6|200.2
7|NA
8|NA
9|200.3
10|201.2
11|201.6
12|201.5
13|201.5
14|203.5
15|204.9
16|207.1
17|NA
18|210.5
19|NA
20|208.8

In [69]:
instance Semigroup (TimeSeries a) where
    (<>) = mergeTimeSeries
    
ts1 <> ts2 <> ts3 <> ts4

1|200.1
2|199.5
3|199.4
4|198.9
5|199.0
6|200.2
7|NA
8|NA
9|200.3
10|201.2
11|201.6
12|201.5
13|201.5
14|203.5
15|204.9
16|207.1
17|210.5
18|210.5
19|NA
20|208.8
21|NA
22|NA
23|NA
24|215.1
25|218.7
26|219.8
27|220.5
28|223.8
29|222.8
30|223.8
31|221.7
32|222.3
33|220.8
34|219.4
35|220.1
36|220.6

In [75]:
instance Monoid (TimeSeries a) where
    mempty = TimeSeries [] []
    
tsAll = mconcat [ts1, ts2, ts3, ts4]
tsAll

1|200.1
2|199.5
3|199.4
4|198.9
5|199.0
6|200.2
7|NA
8|NA
9|200.3
10|201.2
11|201.6
12|201.5
13|201.5
14|203.5
15|204.9
16|207.1
17|210.5
18|210.5
19|NA
20|208.8
21|NA
22|NA
23|NA
24|215.1
25|218.7
26|219.8
27|220.5
28|223.8
29|222.8
30|223.8
31|221.7
32|222.3
33|220.8
34|219.4
35|220.1
36|220.6

In [99]:
-- justValues :: TimeSeries a -> [Maybe a]
justValues (TimeSeries _ v) = filter (/=  Nothing) v

values 

t = justValues tsAll
t
-- :t t
:t tsAll

[Just 200.1,Just 199.5,Just 199.4,Just 198.9,Just 199.0,Just 200.2,Just 200.3,Just 201.2,Just 201.6,Just 201.5,Just 201.5,Just 203.5,Just 204.9,Just 207.1,Just 210.5,Just 210.5,Just 208.8,Just 215.1,Just 218.7,Just 219.8,Just 220.5,Just 223.8,Just 222.8,Just 223.8,Just 221.7,Just 222.3,Just 220.8,Just 219.4,Just 220.1,Just 220.6]

tsAll :: TimeSeries Double

In [105]:
:i length

In [253]:
mean :: (Real a) => [a] -> Double
mean xs = total / count
    where total = (realToFrac . sum) xs
          count = (realToFrac . length) xs

mean [1.2, 1, 2, 3]


1.8

In [117]:
cleanTs :: TimeSeries a -> [a]
cleanTs (TimeSeries k v) = clean
        where 
        just = filter isJust v
        clean = map fromJust just

In [119]:
meanTs :: (Real a) => TimeSeries a -> Maybe Double
meanTs x =   if length clean == 0 
                            then Nothing
                            else Just (mean clean)
    where 
        clean = cleanTs x
        
meanTs (TimeSeries [1] [Nothing])
meanTs tsAll

Nothing

Just 210.5966666666667

In [124]:
maxTs :: (Real a) => TimeSeries a -> Maybe a
maxTs x =   if length clean == 0 
            then Nothing
            else Just (maximum clean)
    where 
        clean = cleanTs x
        
minTs :: (Real a) => TimeSeries a -> Maybe a
minTs x =   if length clean == 0 
            then Nothing
            else Just (minimum clean)
    where 
        clean = cleanTs x

maxTs tsAll
minTs tsAll

Just 223.8

Just 198.9

20.4 Transforming time series

In [131]:
diffPair :: (Num a) => Maybe a -> Maybe a -> Maybe a
diffPair Nothing _ = Nothing
diffPair _ Nothing = Nothing
diffPair (Just a) (Just b) = Just (a - b)

diffPair (Just 2) (Just 3)

Just (-1)

In [139]:

diff :: Num a => TimeSeries a -> TimeSeries a
diff (TimeSeries k v) = TimeSeries k d
    where d = zipWith diffPair v (Nothing : v)

meanTs (diff tsAll)

Just 0.6076923076923071

In [140]:
fromJust Just

: 

In [150]:
meanMaybe :: (Real a) => [Maybe a] -> Maybe Double
meanMaybe xs = if any (== Nothing) xs
        then Nothing
        else (Just avg)
    where avg = mean(map fromJust xs)
    
meanMaybe [(Just 2), (Just 3), Nothing]

Line 1: Redundant bracket
Found:
[(Just 2), (Just 3), Nothing]
Why not:
[Just 2, (Just 3), Nothing]Line 1: Redundant bracket
Found:
[(Just 2), (Just 3), Nothing]
Why not:
[(Just 2), Just 3, Nothing]

Nothing

In [156]:
ma :: (Real a) => [Maybe a] -> Int -> [Maybe Double]
ma [] n = []
ma xs n = if length chunk == n
    then meanMaybe chunk : ma (tail xs) n 
    else []
    where chunk = take n xs
    
ma [Just 1, Just 2, Just 3] 2

[Just 1.5,Just 2.5]

In [166]:
maTS :: (Real a) => TimeSeries a -> Int -> TimeSeries Double
maTS (TimeSeries k v) n = TimeSeries k avg
    where
    pad = replicate (n `div` 2) Nothing
    avg = mconcat [pad, ma v n, pad]
    
maTS tsAll 2

1|NA
2|199.8
3|199.45
4|199.15
5|198.95
6|199.6
7|NA
8|NA
9|NA
10|200.75
11|201.39999999999998
12|201.55
13|201.5
14|202.5
15|204.2
16|206.0
17|208.8
18|210.5
19|NA
20|NA
21|NA
22|NA
23|NA
24|NA
25|216.89999999999998
26|219.25
27|220.15
28|222.15
29|223.3
30|223.3
31|222.75
32|222.0
33|221.55
34|220.10000000000002
35|219.75
36|220.35

 Use the median rather than the mean for smoothing.

In [230]:
median :: (Real a) => [Maybe a] -> Maybe Double
median xs = if (length sorted) == 0
            then Nothing
            else if odd (length sorted)
            then meanMaybe [highVal]
            else meanMaybe [lowVal, highVal]
    where 
    just = filter isJust xs
    sorted = sort just
    halfLen = (length sorted) `div` 2
    lowVal = sorted !! (halfLen-1)
    highVal = sorted !! (halfLen)
    
median [Just 2, Just 3, Nothing]

Just 2.5

In [233]:
ma :: (Real a) => [Maybe a] -> Int -> [Maybe Double]
ma [] n = []
ma xs n = if length chunk == n
    then median chunk : ma (tail xs) n 
    else []
    where chunk = take n xs
    
ma [Just 1, Just 2, Just 3, Just 4, Nothing, Nothing] 2

maTS :: (Real a) => TimeSeries a -> Int -> TimeSeries Double
maTS (TimeSeries k v) n = TimeSeries k avg
    where
    pad = replicate (n `div` 2) Nothing
    avg = mconcat [pad, ma v n, pad]
    
maTS tsAll 5


[Just 1.5,Just 2.5,Just 3.5,Just 4.0,Nothing]

1|NA
2|NA
3|199.4
4|199.4
5|199.2
6|199.0
7|200.2
8|200.3
9|201.2
10|201.35
11|201.5
12|201.5
13|201.6
14|203.5
15|204.9
16|207.1
17|208.8
18|209.65
19|210.5
20|209.65
21|208.8
22|211.95
23|216.89999999999998
24|218.7
25|219.25
26|219.8
27|220.5
28|222.8
29|222.8
30|222.8
31|222.3
32|221.7
33|220.8
34|220.6
35|NA
36|NA

 Create a function that calculates the div rather than the diff of data, capturing the percent change.

Implement a function for calculating the standard deviation of a TS type.

In [265]:
-- std :: TimeSeries Double -> Double
std (TimeSeries k v) = dd
    where
        just = filter isJust v
        clean = map fromJust just
        avg = mean clean
        len = genericLength clean
        meanDiff = map (\x -> x - avg) clean
        upper = sum [(c - avg)^2 | c <- clean]
        dd = sqrt (upper / (len-1))
    
std tsAll

9.605403771273513

If you still want more, the next most useful task is to add and subtract time series from
each other. For each point in the timeline that two TS types have in common, you add or
subtract the values as necessary.

In [274]:
subb :: Num a=>Maybe a -> Maybe a -> Maybe a
subb _ Nothing = Nothing
subb Nothing _ = Nothing
subb (Just a) (Just b) = Just (a - b)


Just (-2)

In [ ]:
subTs (TimeSeries k)